In [ ]:
import ROOT

In [ ]:
# define the C++ functions that are used in the analysis

ROOT.gInterpreter.Declare("""
using ROOT::VecOps::RVec;
using namespace ROOT::Math;

auto makeP4(RVec<Float_t>& pt, RVec<Float_t>& eta, RVec<Float_t>& phi, RVec<Float_t>& mass) {
    std::vector<LorentzVector<PtEtaPhiM4D<float> >> out;
    for(std::size_t i = 0; i < pt.size(); ++i) {
        out.emplace_back(pt[i], eta[i], phi[i], mass[i]);
    }
    return out;
}
""")

In [ ]:
# open the DoubleElectron dataset and declare the analysis

rdf = ROOT.RDataFrame("Events", "../../rdf-analysis/*_DoubleElectron.root");

rdf = rdf.Range(0, 100000) # comment out to treat full DoubleElectron dataset

rdf = rdf.Filter("nElectron == 2")
rdf = rdf.Define("Electron_p4", "makeP4(Electron_pt, Electron_eta, Electron_phi, Electron_mass)")
rdf = rdf.Define("Dielectron_mass", "(Electron_p4[0] + Electron_p4[1]).mass()")
h = rdf.Histo1D(("Dielectron_mass", "Dielectron mass;m_{ee} (GeV);N_{Events}", 1000, 0, 200), "Dielectron_mass")

In [ ]:
# define the fit function, do the fit in a range around the peak and draw the histogram
mass_min = 70
mass_max = 110
# The range of the TF1 is also used for normalizing the PDFs.
# That's why they must be consistent with the fit range, otherwise the result is wrong.
model = ROOT.TF1("model", "NSUM(expo, gaus)", mass_min, mass_max)

n_events = rdf.Count().GetValue()
model.SetParameters(n_events * 0.5, n_events * 0.5, -0.02, 90, 30)

h.Fit(model, "", "", mass_min, mass_max)

h.Draw("E") # draw with error bars

ROOT.gStyle.SetOptFit(1111) # include the fit result in the plot

ROOT.gPad.Draw()